# Classification of COVID-19 based on CT scans using CNN and GA

# Loading dependencies

In [44]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
from keras.utils import plot_model
from keras import models 
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_curve, auc, precision_score, accuracy_score, confusion_matrix
from keras.layers import BatchNormalization
from keras.models import Model

# Loading dataset

### Image Read and Resize Function

In [14]:
def read_image(filepath):
    return cv2.imread(os.path.join(data_dir, filepath)) 

def resize_image(image, image_size):
    return cv2.resize(image.copy(), image_size, interpolation = cv2.INTER_AREA)

In [15]:
disease_types=['COVID', 'non-COVID']
data_dir = '../input/sarscov2-ctscan-dataset/'
train_dir = os.path.join(data_dir)

In [16]:
train_data = []
for defects_id, sp in enumerate(disease_types):
    for file in os.listdir(os.path.join(train_dir, sp)):
        train_data.append(['{}/{}'.format(sp, file), defects_id, sp])      
train = pd.DataFrame(train_data, columns=['File', 'DiseaseID','Disease Type'])

train.head()

### Randomize the order of training set

In [17]:
SEED = 42
train = train.sample(frac=1, random_state=SEED) 
train.index = np.arange(len(train)) # Reset indices
train.head()

### Display images of COVID

In [18]:
def plot_defects(defect_types, rows, cols):
    fig, ax = plt.subplots(rows, cols, figsize=(8, 8))
    defect_files = train['File'][train['Disease Type'] == defect_types].values
    n = 0
    for i in range(rows):
        for j in range(cols):
            image_path = os.path.join(data_dir, defect_files[n])
            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])
            ax[i, j].imshow(cv2.imread(image_path))
            n += 1
# Displays first n images of class from training set
plot_defects('COVID', 3, 3)

### Display images of non-COVID

In [19]:
plot_defects('non-COVID', 3,3)

In [20]:
IMAGE_SIZE = 64
X = np.zeros((train.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
for i, file in tqdm(enumerate(train['File'].values), total = len(train)):
    image = read_image(file)
    if image is not None:
        X[i] = resize_image(image, (IMAGE_SIZE, IMAGE_SIZE))
X /= 255.        
y = train['DiseaseID'].values
print(X.shape)
print(y.shape)

## Train Test split

In [21]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = 1)

# CNN Model

In [37]:
np.random.seed(3)

filepath = "cnn_model"

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPool2D((2, 2)))


model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Dropout(0.3))


model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

#FC1
model.add(layers.Dense(128 , activation='relu'))

#FC2
model.add(layers.Dense(100,name ='feature_dense' , activation='relu'))


#output FC
model.add(layers.Dense(1, activation='sigmoid'))

In [38]:
plot_model(model, 
           show_shapes = True, 
           show_layer_names = True, 
           rankdir = 'TB', 
           expand_nested = False, 
           dpi = 60)

In [39]:
model.summary()

# Training the model

In [40]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=64, epochs=10,
                    verbose=1,validation_split=0.1)

In [41]:
y_pred = model.predict_classes(x_test)

In [42]:
model.evaluate(x_test,y_test)

### Extract feature using intermediate model till 'feature_dense' layer for input data

In [45]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('feature_dense').output)
intermediate_layer_model.summary()

*The output layer dimension is 100, so if we predict using this netowork on a new data with same input dimension, we can create 100 new complex features.*

In [56]:
featurs = intermediate_layer_model.predict(x_train)
features = pd.DataFrame(featurs)
print('feauture_data shape:', features.shape)
features.head(5) 

In [47]:
y_df = pd.DataFrame(y_train)
y_df = y_df.rename(columns = { 0 : "result",})
y_df.head()

In [ ]:
#get Feautre vectors 
# X_train_features=model.predict(x_train)
# X_test_features=model.predict(x_test)

In [51]:
result = pd.concat([features, y_df], axis=1)
result.head()

## Feature selection using GA

In [52]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
from sklearn.preprocessing import MinMaxScaler
import sys
from pandas import DataFrame


def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def getFitness(individual, X, y):
    """
    Feature subset fitness function
    """

    if(individual.count(0) != len(individual)):
        # get index with value 0
        cols = [index for index in range(
            len(individual)) if individual[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = LogisticRegression(max_iter=2000)

        return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
    else:
        return(0,)


def geneticAlgorithm(X, y, n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,toolbox.individual)
    toolbox.register("evaluate", getFitness, X=X, y=y)              # fitness function 
    toolbox.register("mate", tools.cxOnePoint)                      # single point cross over 
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)        # flip bits , mutation 
    toolbox.register("select", tools.selTournament, tournsize=3)    # tornament selection 

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof


def bestIndividual(hof, X, y):
    """
    Get the best individual
    """
    maxAccurcy = 0.0
    for individual in hof:
        if(individual.fitness.values[0] > maxAccurcy):
            maxAccurcy = individual.fitness.values
            _individual = individual

    _individualHeader = [list(X)[i] for i in range(
        len(_individual)) if _individual[i] == 1]
    return _individual.fitness.values, _individual, _individualHeader


if __name__ == '__main__':
    
    # initialize 

#     dataframePath = 'features.csv'
    n_pop = 20 
    n_gen = 2 

    # read dataframe from csv
#     df = pd.read_csv(dataframePath, sep=',')
    df = result

    # encode labels column to numbers
    le = LabelEncoder()
    le.fit(df.iloc[:, -1])
    y = le.transform(df.iloc[:, -1])
    X = df.iloc[:, :-1]
    # scaler = MinMaxScaler()
    # X = scaler.fit_transform(X)
    # X = DataFrame(X)


    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    print("Accuracy with all features: \t" +
          str(getFitness(individual, X, y)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(X, y, n_pop, n_gen)

    # select the best individual
    accuracy, individual, header = bestIndividual(hof, X, y)
    print('Best Accuracy: \t' + str(accuracy))
    print('Number of Features in Subset: \t' + str(individual.count(1)))
    print('Individual: \t\t' + str(individual))
    print('Feature Subset\t: ' + str(header))


    # classifier 

    # print('\n\ncreating a new classifier with the result')

    # # read dataframe from csv one more time
    # df = pd.read_csv(dataframePath, sep=',')

    # # with feature subset
    # X = df[header]

    # clf = LogisticRegression(max_iter=1000)

    # scores = cross_val_score(clf, X, y, cv=5)
    # print("Accuracy with Feature Subset: \t" + str(avg(scores)) + "\n")

# Classification 


In [53]:
# classifier 

print('\n\ncreating a new classifier with the result')

# read dataframe from csv one more time
df = result

# with feature subset
X = df[header]

clf = LogisticRegression(max_iter=2000)

scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy with Feature Subset: \t" + str(avg(scores)) + "\n")

# ROC curve

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_pred)
    roc_auc[i] = auc(fpr[i], tpr[i])

    
plt.figure()
plt.plot(fpr[1], tpr[1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.show()

In [54]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Performence metrics

In [55]:
sp = tn/(tn+fp)
sn = tp/(tp+fn)

print('f1 score =  %.3f'%f1_score(y_test, y_pred))
print('Precision =  %.3f'%precision_score(y_test, y_pred))
print('Test accuracy =  %.3f'%accuracy_score(y_test, y_pred))
print('Specificity =  %.3f'%sp)
print('Sensitivity =  %.3f'%sn)